In [14]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'
SINCE_DATE = int(datetime(2017, 1, 1).timestamp())
UNTIL_DATE = int(datetime.now().timestamp())

# URLs for fetching data
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/price_usd_close',
    'https://api.glassnode.com/v1/metrics/market/mvrv'
]

# List of all assets (split into chunks for readability)
ASSETS = [
    'BTC', 'ETH', 'LTC', '1INCH', 'AAVE', 'ABT', 'ACH', 'ACX'
]

def fetch_glassnode_data(url, asset='BTC'):
    """
    Fetches data from Glassnode API for a specific metric and asset.
    """
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        column_name = f"{metric_name}_{asset}"
        df.columns = ['t', column_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {asset} from {url}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error processing data for {asset} from {url}: {e}")
        return None

def fetch_all_data():
    """
    Fetches data for all metrics and assets and combines them into a single DataFrame.
    """
    all_dfs = []
    for metric_url in METRICS:
        for asset in ASSETS:
            print(f"Fetching {metric_url.split('/')[-1]} for {asset}")
            df = fetch_glassnode_data(metric_url, asset)
            if df is not None:
                all_dfs.append(df)

    if not all_dfs:
        raise Exception("No data was successfully fetched")

    merged_df = all_dfs[0]
    for df in all_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='t', how='outer')

    merged_df.set_index('t', inplace=True)
    return merged_df

try:
    final_df = fetch_all_data()
    print("Data collection complete")
    print(f"DataFrame shape: {final_df.shape}")
    print("\nColumns in the dataset:")
    print(final_df.columns.tolist())
except Exception as e:
    print(f"Error in data collection process: {e}")

Fetching price_usd_close for BTC
Fetching price_usd_close for ETH
Fetching price_usd_close for LTC
Fetching price_usd_close for 1INCH
Fetching price_usd_close for AAVE
Fetching price_usd_close for ABT
Fetching price_usd_close for ACH
Fetching price_usd_close for ACX
Fetching mvrv for BTC
Fetching mvrv for ETH
Fetching mvrv for LTC
Fetching mvrv for 1INCH
Fetching mvrv for AAVE
Fetching mvrv for ABT
Fetching mvrv for ACH
Fetching mvrv for ACX
Data collection complete
DataFrame shape: (2853, 16)

Columns in the dataset:
['price_usd_close_BTC', 'price_usd_close_ETH', 'price_usd_close_LTC', 'price_usd_close_1INCH', 'price_usd_close_AAVE', 'price_usd_close_ABT', 'price_usd_close_ACH', 'price_usd_close_ACX', 'mvrv_BTC', 'mvrv_ETH', 'mvrv_LTC', 'mvrv_1INCH', 'mvrv_AAVE', 'mvrv_ABT', 'mvrv_ACH', 'mvrv_ACX']


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2853 entries, 2017-01-01 to 2024-10-23
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price_usd_close_BTC    2853 non-null   float64
 1   price_usd_close_ETH    2853 non-null   float64
 2   price_usd_close_LTC    2853 non-null   float64
 3   price_usd_close_1INCH  1399 non-null   float64
 4   price_usd_close_AAVE   1483 non-null   float64
 5   price_usd_close_ABT    1833 non-null   float64
 6   price_usd_close_ACH    1512 non-null   float64
 7   price_usd_close_ACX    696 non-null    float64
 8   mvrv_BTC               2853 non-null   float64
 9   mvrv_ETH               2853 non-null   float64
 10  mvrv_LTC               2853 non-null   float64
 11  mvrv_1INCH             1399 non-null   float64
 12  mvrv_AAVE              1483 non-null   float64
 13  mvrv_ABT               1833 non-null   float64
 14  mvrv_ACH               1512 non-null  

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

def plot_mvrv_heatmap(df):
    """
    Plots a heatmap of MVRV values for all assets over time.
    
    Parameters:
    df (pandas.DataFrame): A DataFrame containing the MVRV data, with columns for timestamps and asset names.
    """
    # Filter the DataFrame to only include MVRV metrics
    mvrv_metrics = df.filter(regex='mvrv', axis=1)
    
    # Convert the DataFrame to a format suitable for plotting
    data = mvrv_metrics.reset_index()
    data = data.melt(id_vars='index', var_name='Asset', value_name='MVRV')
    data['Date'] = data['index'].dt.strftime('%Y-%m-%d')
    
    # Ensure all columns have the same length by filling missing values with NaN
    max_length = max(len(data[col]) for col in data.columns)
    for col in data.columns:
        if len(data[col]) < max_length:
            data[col] = data[col].reindex(range(max_length)).fillna(method='ffill')
    
    # Create the heatmap
    pivot_table = data.pivot(index='Date', columns='Asset', values='MVRV')
    fig, ax = plt.subplots(figsize=(12, 6))
    im = ax.imshow(pivot_table.values, cmap='coolwarm', aspect='auto')
    
    # Add labels and formatting
    ax.set_xticks(np.arange(len(pivot_table.columns)))
    ax.set_xticklabels(pivot_table.columns, rotation=90)
    ax.set_yticks(np.arange(len(pivot_table.index)))
    ax.set_yticklabels(pivot_table.index)
    ax.set_title('MVRV Heatmap')
    ax.set_xlabel('Asset')
    ax.set_ylabel('Date')
    
    # Add a colorbar
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel('MVRV', rotation=-90, va="bottom")
    
    plt.show()

# Example usage
df = pd.DataFrame({
    'index': pd.date_range(start=datetime(2023, 1, 1), end=datetime(2023, 6, 30), freq='D'),
    'BTC_mvrv': np.random.uniform(0.8, 1.2, size=182),
    'ETH_mvrv': np.random.uniform(0.7, 1.1, size=180),
    'LTC_mvrv': np.random.uniform(0.6, 1.0, size=181)
})

plot_mvrv_heatmap(df)

ValueError: All arrays must be of the same length